# Neural Machine Translation

- Translate a given sentence in one language to another desired language.

#### In this notebook, we aim to build a model which can translate German sentences to English.

## Dataset

Dataset is taken from http://www.manythings.org/anki/.

We are considering German – English deu-eng.zip file from the above mentioned website.

In the above zip file there is a file with name **`deu.txt`** that contains **152,820** pairs of English to German phrases, one pair per line with a tab separating the phrases.


For example,

The first 5 lines in deu.txt are as given below.

***
```
Hi.    Hallo!
Hi.    Grüß Gott!
Run!    Lauf!
Wow!    Potzdonner!
Wow!    Donnerwetter!
```
***


## Problem

### Given a sequence of words in German as input, predict the sequence of words in English.

### 1. Prepare Data


The preprocessing of the data involves:

1. Removing punctuation marks from the data.

2. Converting text corpus into lower case characters.

3. Split into Train and Test sets.

4. Shuffling the sentences.



The above tasks are done  and full dataset is given as **``english-german-both.pkl``** respectively.

Download dataset files from here: https://drive.google.com/open?id=1gWVk7SuuE93Cf_nT9Lb7GBCiwfAgdBiX




# Character level Machine Translation

## Initialize parameters
Run the below code to initialize the variables required for the model.

In [0]:
batch_size = 64  # Batch size for training.
epochs = 10  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

### Connect to google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


### Give the path for the folder in which the dataset is present in google drive

In [0]:
project_path = "/content/drive/My Drive/Great Learning Residency 10.2 Sequence Models in NLP/Lab10/"

### Change present working directory

In [0]:
import os
os.chdir(project_path)

## Load the pickle file (`english-german-both.pkl`) into a variable with name `dataset`
Run the below code to load the .pkl file.

In [0]:
import pickle

with open(project_path + 'english-german-both.pkl', 'rb') as f:
  dataset = pickle.load(f)

## Check the `dataset` variable at this step. It should be as given below

In [6]:
dataset

array([['stay with us', 'bleib bei uns'],
       ['she wants him', 'sie will ihn'],
       ['youre strong', 'du bist stark'],
       ...,
       ['i thought so', 'das dachte ich mir'],
       ['keep warm', 'haltet euch warm'],
       ['im sick', 'ich bin krank']], dtype='<U291')

## Feature set and target set division from the **dataset**

### Run the below code to divide the dataset into feature set(input) and target set(output). 

1. We are creating two lists for storing input sentences and output sentences separately. 
2. We are storing each character in a list from both input and target sets separately. 
3. Print and check `input_texts` and `target_texts`.
4. Print and check `input_characters` and `target_characters`.

In [0]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

for line in dataset[: min(num_samples, len(dataset) - 1)]:
    input_text, target_text = line[0], line[1]
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

### Print input text

In [8]:
print("THE INPUT TEXT IS:",input_texts)

THE INPUT TEXT IS: ['stay with us', 'she wants him', 'youre strong', 'examine this', 'heres my card', 'tom burped', 'it is no joke', 'tom is a spy', 'im a teenager', 'im not crazy', 'we cant do it', 'i like sweets', 'put it down', 'tom looks fine', 'i am hungarian', 'youre dying', 'go away', 'tom denied this', 'yes of course', 'this is ugly', 'tom cried again', 'ill prove it', 'do as he says', 'please tell me', 'he had ambition', 'look here', 'cuff him', 'is it far away', 'were cousins', 'is tom dreaming', 'i envy tom', 'find a job', 'im the expert', 'just try it', 'theyll find me', 'a car hit tom', 'have tom do it', 'close your eyes', 'come on in here', 'i like this', 'tom is history', 'try it on', 'ill sleep here', 'does that help', 'tom moved away', 'well be ok', 'im on my own', 'toms out cold', 'eat everything', 'you said it', 'he looked young', 'dont insult me', 'im upset', 'you never ask', 'i broke it', 'theyre happy', 'tom seemed sad', 'tom is a loser', 'its up to you', 'are the

### Print target text

In [9]:
print("THE TARGET TEXT IS:",target_texts)

THE TARGET TEXT IS: ['\tbleib bei uns\n', '\tsie will ihn\n', '\tdu bist stark\n', '\tuntersuchen sie das\n', '\thier ist meine karte\n', '\ttom stie auf\n', '\tdas ist kein witz\n', '\ttom ist ein spion\n', '\tich bin ein teenager\n', '\tich bin nicht verruckt\n', '\twir konnen das nicht\n', '\tich mag sues\n', '\tlegt es hin\n', '\ttom sieht gut aus\n', '\tich bin ungarin\n', '\tdu stirbst\n', '\tverkrumele dich\n', '\ttom hat das bestritten\n', '\tja naturlich\n', '\tdas ist grauslich\n', '\ttom hat wieder geweint\n', '\tich werde es beweisen\n', '\ttu was er sagt\n', '\tbitte sagt es mir\n', '\ter war strebsam\n', '\tschau her\n', '\tlegen sie ihm handschellen an\n', '\tist es weit weg\n', '\twir sind cousins\n', '\tist tom am traumen\n', '\tich beneide tom\n', '\tsuch dir einen job\n', '\tich bin der mann vom fach\n', '\tversuchs doch einfach\n', '\tsie werden mich finden\n', '\ttom wurde von einem auto angefahren\n', '\tlass es tom machen\n', '\tmacht die augen zu\n', '\tkomm her

### Print input character

In [10]:
print("THE INPUT CHARACTERS ARE:",input_characters)

THE INPUT CHARACTERS ARE: {'x', 'v', 'c', 'f', 't', 'g', 'z', 'b', 'e', 'h', ' ', 'w', 'a', 'u', 'q', 'i', 'l', 'y', 'j', 'r', 'k', 'o', 'm', 'n', 's', 'd', 'p'}


### Print target character

In [11]:
print("THE TARGER CHARACTERS ARE:",target_characters)

THE TARGER CHARACTERS ARE: {'v', 'x', '\t', 'c', 'f', 't', 'g', 'z', 'b', 'e', 'h', ' ', 'w', 'a', 'u', 'q', 'i', 'l', 'j', 'y', 'r', 'k', 'o', 'm', 'n', 's', 'd', '\n', 'p'}


## Stats from the dataset

### Run the below code to check the stats from the dataset

In [0]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [13]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 9999
Number of unique input tokens: 27
Number of unique output tokens: 29
Max sequence length for inputs: 15
Max sequence length for outputs: 51


## Build character to index dictionary names `input_token_index` and `target_token_index` for input and target sets respectively.

In [0]:
input_token_index = dict(
  [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
  [(char, i) for i, char in enumerate(target_characters)])
# https://keras.io/preprocessing/text/ for tokenizer

### Print input_index_token

In [15]:
input_token_index

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

### Print target_token_index

In [16]:
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 'a': 3,
 'b': 4,
 'c': 5,
 'd': 6,
 'e': 7,
 'f': 8,
 'g': 9,
 'h': 10,
 'i': 11,
 'j': 12,
 'k': 13,
 'l': 14,
 'm': 15,
 'n': 16,
 'o': 17,
 'p': 18,
 'q': 19,
 'r': 20,
 's': 21,
 't': 22,
 'u': 23,
 'v': 24,
 'w': 25,
 'x': 26,
 'y': 27,
 'z': 28}

## Build Model
Initialize the required layers from keras

### Import libraries

In [17]:
from __future__ import print_function
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

Using TensorFlow backend.


### Run the below code to build one-hot vectors for the characters

In [0]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [0]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

### Build the encoder Model

Define an input sequence and process it.

Discard `encoder_outputs` and only keep the states.

In [20]:
# Defining an input sequence and processing it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
# latent_dim = 256  # Latent dimensionality of the encoding space.
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
#Discarding `encoder_outputs` and only keeping the states.
encoder_states = [state_h, state_c]
encoder_model = Model(encoder_inputs, encoder_states)

### Build the decoder Model

Set up the decoder, using `encoder_states` as initial state.

We set up our decoder to return full output sequences, and to return internal states as well. We don't use the return states in the training model, but we will use them in inference.

In [0]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

### Define Model

Define the model that will turn `encoder_input_data ` & ` decoder_input_data` into `decoder_target_data`

In [0]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

### Compile and fit the model

In [23]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=25,
          validation_split=0.2)
#In order to get the appropriate number of epochs we should try getting the loss and validation loss as similar as possible




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 7999 samples, validate on 2000 samples
Epoch 1/25





7999/7999 [==============================] - 11s 1ms/step - loss: 0.9525 - val_loss: 0.8055
Epoch 2/25
7999/7999 [==============================] - 10s 1ms/step - loss: 0.7421 - val_loss: 0.6729
Epoch 3/25
7999/7999 [==============================] - 10s 1ms/step - loss: 0.6442 - val_loss: 0.6014
Epoch 4/25
7999/7999 [==============================] - 10s 1ms/step - loss: 0.5875 - val_loss: 0.5598
Epoch 5/25
7999/7999 [==============================] - 10s 1ms/step - loss: 0.5482 - val_loss: 0.5275
Epoch 6/25
7999/7999 [==============================] - 10s 1ms/step - loss: 0.5195 - val_loss: 0.5087
Epoch 7/25
7999/7999 [==============================] - 10s 1ms/step - loss: 0.4958 - val_loss: 0.4919
Epoch 8/25
7999/7999 [==============================] - 10s 1ms/step - loss: 0.4749 - val_loss: 0.4705
Epoch 9/25
7999/7999 [==

### Save the model

In [0]:
# Save model
model.save('s2s.h5')

## Run the below code for inferencing the model

In [0]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

## Reverse-lookup token index to decode sequences back to something readable.

In [0]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [27]:
print(reverse_input_char_index)
print(reverse_target_char_index)

{0: ' ', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}
{0: '\t', 1: '\n', 2: ' ', 3: 'a', 4: 'b', 5: 'c', 6: 'd', 7: 'e', 8: 'f', 9: 'g', 10: 'h', 11: 'i', 12: 'j', 13: 'k', 14: 'l', 15: 'm', 16: 'n', 17: 'o', 18: 'p', 19: 'q', 20: 'r', 21: 's', 22: 't', 23: 'u', 24: 'v', 25: 'w', 26: 'x', 27: 'y', 28: 'z'}


In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

## Run the below code for checking some outputs from the model.

In [29]:
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: stay with us
Decoded sentence: bleib duruck

-
Input sentence: she wants him
Decoded sentence: sie war nicht gelangt

-
Input sentence: youre strong
Decoded sentence: du bist nicht der gefallen

-
Input sentence: examine this
Decoded sentence: entschuldigt es

-
Input sentence: heres my card
Decoded sentence: hier ist dei ernstatte

-
Input sentence: tom burped
Decoded sentence: tom hat geschlafen

-
Input sentence: it is no joke
Decoded sentence: es ist nicht danken

-
Input sentence: tom is a spy
Decoded sentence: tom ist nicht geschnitten

-
Input sentence: im a teenager
Decoded sentence: ich bin nicht beschaftigt

-
Input sentence: im not crazy
Decoded sentence: ich bin nicht beschaftigt

